In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1044939,2021-03-01,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1044940,2021-03-02,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1044941,2021-03-03,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1044942,2021-03-04,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
28307,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28309,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28311,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28313,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28315,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
646885,2021-03-01,Arkansas,Arkansas,5001,1993,33.00,5000,Arkansas,AR,Arkansas,State,3017804
646887,2021-03-02,Arkansas,Arkansas,5001,1996,33.00,5000,Arkansas,AR,Arkansas,State,3017804
646889,2021-03-03,Arkansas,Arkansas,5001,1997,33.00,5000,Arkansas,AR,Arkansas,State,3017804
646891,2021-03-04,Arkansas,Arkansas,5001,1999,33.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1044939,2021-03-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1044940,2021-03-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1044941,2021-03-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1044942,2021-03-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-05') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
409,2021-03-05,Snohomish,Washington,31299,546.00,53061,WA,County,822083,3807.28,66.42
816,2021-03-05,Cook,Illinois,477381,9983.00,17031,IL,County,5150233,9269.11,193.84
1222,2021-03-05,Orange,California,262241,4075.00,6059,CA,County,3175692,8257.76,128.32
1627,2021-03-05,Maricopa,Arizona,514132,9286.00,4013,AZ,County,4485414,11462.31,207.03
2032,2021-03-05,Los Angeles,California,1200764,21910.00,6037,CA,County,10039107,11960.86,218.25
...,...,...,...,...,...,...,...,...,...,...,...
1044491,2021-03-05,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1044635,2021-03-05,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1044748,2021-03-05,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1044857,2021-03-05,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
409,2021-03-05,Snohomish,Washington,31299,546.00,53061,WA,County,822083,3807.28,66.42,66.42,3807.28
816,2021-03-05,Cook,Illinois,477381,9983.00,17031,IL,County,5150233,9269.11,193.84,193.84,9269.11
1222,2021-03-05,Orange,California,262241,4075.00,6059,CA,County,3175692,8257.76,128.32,128.32,8257.76
1627,2021-03-05,Maricopa,Arizona,514132,9286.00,4013,AZ,County,4485414,11462.31,207.03,207.03,11462.31
2032,2021-03-05,Los Angeles,California,1200764,21910.00,6037,CA,County,10039107,11960.86,218.25,218.25,11960.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044491,2021-03-05,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1044635,2021-03-05,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1044748,2021-03-05,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1044857,2021-03-05,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
735897,2021-03-05,Clarke,Alabama,3445,53.00,1025,AL,County,23622,14583.86,224.37,224.37,14583.86,1
676302,2021-03-05,Hale,Alabama,2122,69.00,1065,AL,County,14651,14483.65,470.96,470.96,14483.65,2
529581,2021-03-05,Lowndes,Alabama,1328,51.00,1085,AL,County,9726,13654.12,524.37,524.37,13654.12,3
469729,2021-03-05,Etowah,Alabama,13407,325.00,1055,AL,County,102268,13109.67,317.79,317.79,13109.67,4
529926,2021-03-05,Marengo,Alabama,2430,57.00,1091,AL,County,18863,12882.36,302.18,302.18,12882.36,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645524,2021-03-05,Converse,Wyoming,970,16.00,56009,WY,County,13822,7017.80,115.76,115.76,7017.80,19
645867,2021-03-05,Sublette,Wyoming,679,7.00,56035,WY,County,9831,6906.72,71.20,71.20,6906.72,20
891805,2021-03-05,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,21
801809,2021-03-05,Lincoln,Wyoming,1248,11.00,56023,WY,County,19830,6293.49,55.47,55.47,6293.49,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
529581,2021-03-05,Lowndes,Alabama,1328,51.00,1085,AL,County,9726,13654.12,524.37,524.37,13654.12,3,1
676302,2021-03-05,Hale,Alabama,2122,69.00,1065,AL,County,14651,14483.65,470.96,470.96,14483.65,2,2
468345,2021-03-05,Clay,Alabama,1460,54.00,1027,AL,County,13235,11031.36,408.01,408.01,11031.36,20,3
225726,2021-03-05,Walker,Alabama,6593,259.00,1127,AL,County,63521,10379.24,407.74,407.74,10379.24,31,4
528891,2021-03-05,Crenshaw,Alabama,1465,55.00,1041,AL,County,13772,10637.53,399.36,399.36,10637.53,30,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569601,2021-03-05,Hot Springs,Wyoming,361,3.00,56017,WY,County,4413,8180.38,67.98,67.98,8180.38,17,19
781926,2021-03-05,Uinta,Wyoming,2070,12.00,56041,WY,County,20226,10234.35,59.33,59.33,10234.35,4,20
801809,2021-03-05,Lincoln,Wyoming,1248,11.00,56023,WY,County,19830,6293.49,55.47,55.47,6293.49,22,21
224670,2021-03-05,Teton,Wyoming,3375,9.00,56039,WY,County,23464,14383.74,38.36,38.36,14383.74,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3807.28,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3807.28,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3807.28,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3807.28,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.42,3807.28,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1042475,2021-03-01,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1042476,2021-03-02,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1042477,2021-03-03,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1042478,2021-03-04,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
733095,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14583.86,39,1,2.00
733096,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14583.86,39,1,0.00
733097,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,224.37,14583.86,39,1,1.00
733098,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,224.37,14583.86,39,1,4.00
733099,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,224.37,14583.86,39,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
815133,2021-03-01,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5511.60,7,23,0.00
815134,2021-03-02,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,1.00
815135,2021-03-03,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
815136,2021-03-04,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
527459,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13654.12,1,3,1.00,0.00
527460,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13654.12,1,3,0.00,0.00
527461,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13654.12,1,3,0.00,0.00
527462,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13654.12,1,3,0.00,0.00
527463,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13654.12,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525729,2021-03-01,Albany,Wyoming,3909,11.00,56001,WY,County,38880,10054.01,28.29,28.29,10064.30,23,6,4.00,0.00
525730,2021-03-02,Albany,Wyoming,3911,11.00,56001,WY,County,38880,10059.16,28.29,28.29,10064.30,23,6,2.00,0.00
525731,2021-03-03,Albany,Wyoming,3911,11.00,56001,WY,County,38880,10059.16,28.29,28.29,10064.30,23,6,0.00,0.00
525732,2021-03-04,Albany,Wyoming,3912,11.00,56001,WY,County,38880,10061.73,28.29,28.29,10064.30,23,6,1.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-05') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
269342,2021-03-05,Imperial,California,26946,638.00,6025,CA,County,181215,14869.63,352.07,352.07,14869.63,1,2,7.00,3.00
2032,2021-03-05,Los Angeles,California,1200764,21910.00,6037,CA,County,10039107,11960.86,218.25,218.25,11960.86,2,5,2027.00,132.00
471065,2021-03-05,Inyo,California,1300,37.00,6027,CA,County,18039,7206.61,205.11,205.11,7206.61,3,27,6.00,0.00
60177,2021-03-05,Stanislaus,California,56602,957.00,6099,CA,County,550660,10278.94,173.79,173.79,10278.94,4,10,80.00,2.00
74574,2021-03-05,Tulare,California,48203,770.00,6107,CA,County,466195,10339.66,165.17,165.17,10339.66,5,9,40.00,2.00
27942,2021-03-05,Riverside,California,290904,3892.00,6065,CA,County,2470546,11774.89,157.54,157.54,11774.89,6,6,160.00,8.00
47905,2021-03-05,San Joaquin,California,67284,1182.00,6077,CA,County,762148,8828.21,155.09,155.09,8828.21,7,18,71.00,5.00
427013,2021-03-05,Merced,California,29376,413.00,6047,CA,County,277680,10579.08,148.73,148.73,10579.08,8,8,51.00,1.00
570920,2021-03-05,Kings,California,22150,227.00,6031,CA,County,152940,14482.80,148.42,148.42,14482.80,9,3,31.00,0.00
27214,2021-03-05,Fresno,California,96023,1466.00,6019,CA,County,999101,9610.94,146.73,146.73,9610.94,10,14,130.00,8.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
992851,2021-03-05,Lassen,California,5623,19.00,6035,CA,County,30573,18392.05,62.15,62.15,18392.05,38,1,0.00,0.00
269342,2021-03-05,Imperial,California,26946,638.00,6025,CA,County,181215,14869.63,352.07,352.07,14869.63,1,2,7.00,3.00
570920,2021-03-05,Kings,California,22150,227.00,6031,CA,County,152940,14482.80,148.42,148.42,14482.80,9,3,31.00,0.00
134336,2021-03-05,San Bernardino,California,287513,3116.00,6071,CA,County,2180085,13188.16,142.93,142.93,13188.16,11,4,267.00,94.00
2032,2021-03-05,Los Angeles,California,1200764,21910.00,6037,CA,County,10039107,11960.86,218.25,218.25,11960.86,2,5,2027.00,132.00
27942,2021-03-05,Riverside,California,290904,3892.00,6065,CA,County,2470546,11774.89,157.54,157.54,11774.89,6,6,160.00,8.00
168813,2021-03-05,Kern,California,104057,937.00,6029,CA,County,900202,11559.29,104.09,104.09,11559.29,17,7,163.00,13.00
427013,2021-03-05,Merced,California,29376,413.00,6047,CA,County,277680,10579.08,148.73,148.73,10579.08,8,8,51.00,1.00
74574,2021-03-05,Tulare,California,48203,770.00,6107,CA,County,466195,10339.66,165.17,165.17,10339.66,5,9,40.00,2.00
60177,2021-03-05,Stanislaus,California,56602,957.00,6099,CA,County,550660,10278.94,173.79,173.79,10278.94,4,10,80.00,2.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
269342,2021-03-05,Imperial,California,26946,638.00,6025,CA,County,181215,14869.63,352.07,352.07,14869.63,1,2,7.00,3.00
2032,2021-03-05,Los Angeles,California,1200764,21910.00,6037,CA,County,10039107,11960.86,218.25,218.25,11960.86,2,5,2027.00,132.00
471065,2021-03-05,Inyo,California,1300,37.00,6027,CA,County,18039,7206.61,205.11,205.11,7206.61,3,27,6.00,0.00
60177,2021-03-05,Stanislaus,California,56602,957.00,6099,CA,County,550660,10278.94,173.79,173.79,10278.94,4,10,80.00,2.00
74574,2021-03-05,Tulare,California,48203,770.00,6107,CA,County,466195,10339.66,165.17,165.17,10339.66,5,9,40.00,2.00
27942,2021-03-05,Riverside,California,290904,3892.00,6065,CA,County,2470546,11774.89,157.54,157.54,11774.89,6,6,160.00,8.00
47905,2021-03-05,San Joaquin,California,67284,1182.00,6077,CA,County,762148,8828.21,155.09,155.09,8828.21,7,18,71.00,5.00
427013,2021-03-05,Merced,California,29376,413.00,6047,CA,County,277680,10579.08,148.73,148.73,10579.08,8,8,51.00,1.00
570920,2021-03-05,Kings,California,22150,227.00,6031,CA,County,152940,14482.80,148.42,148.42,14482.80,9,3,31.00,0.00
27214,2021-03-05,Fresno,California,96023,1466.00,6019,CA,County,999101,9610.94,146.73,146.73,9610.94,10,14,130.00,8.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,352.07,14869.63,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,352.07,14869.63,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,352.07,14869.63,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,352.07,14869.63,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,352.07,14869.63,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3522,03/01/21,Lassen,5601,19.00,18320.09,62.15,62.15,18392.05,38,1,0.00,0.00
3523,03/02/21,Lassen,5601,19.00,18320.09,62.15,62.15,18392.05,38,1,0.00,0.00
3524,03/03/21,Lassen,5623,19.00,18392.05,62.15,62.15,18392.05,38,1,22.00,0.00
3525,03/04/21,Lassen,5623,19.00,18392.05,62.15,62.15,18392.05,38,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,352.07,14869.63,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,352.07,14869.63,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,352.07,14869.63,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,352.07,14869.63,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,352.07,14869.63,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3596,03/01/21,Fresno,95548,1443.00,9563.40,144.43,146.73,9610.94,10,14,155.00,0.00
3597,03/02/21,Fresno,95677,1451.00,9576.31,145.23,146.73,9610.94,10,14,129.00,8.00
3598,03/03/21,Fresno,95785,1451.00,9587.12,145.23,146.73,9610.94,10,14,108.00,0.00
3599,03/04/21,Fresno,95893,1458.00,9597.93,145.93,146.73,9610.94,10,14,108.00,7.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)